In [3]:
import numpy as np
import pandas as pd
import re

In [4]:
def get_sites(string, site_type='ACT_SITE'):
    """string=site entry
    Returns the sites as a list, e.g. if 
    string = 'ACT_SITE 85;  /evidence=...; ACT_SITE 110;  /evidence=...'
    returns [85, 110]"""
    
    pattern = re.compile(f'{site_type} [0-9]+;')
    matches = pattern.findall(string)
    get_site = lambda match: int(match.split(' ')[1][:-1])
    return [get_site(match) for match in matches]
get_binding_sites = lambda string: get_sites(string, 'BINDING')

In [5]:
df = pd.read_csv('uniprot.tab', sep='\t', lineterminator='\n')
df = df.fillna('')

In [6]:
df[:5]

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Active site,Binding site
0,Q9SJT1,SAE2_ARATH,reviewed,SUMO-activating enzyme subunit 2 (EC 2.3.2.-) ...,SAE2 EMB2764 At2g21470 F3K23.23,Arabidopsis thaliana (Mouse-ear cress),700,MATQQQQSAIKGAKVLMVGAGGIGCELLKTLALSGFEDIHIIDMDT...,"ACT_SITE 168; /note=""Glycyl thioester interme...","BINDING 43; /note=""ATP""; /evidence=""ECO:0000..."
1,P56221,SCYD_MAGO7,reviewed,Scytalone dehydratase (SD) (SDH) (EC 4.2.1.94),SDH1 MGG_05059,Magnaporthe oryzae (strain 70-15 / ATCC MYA-46...,172,MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTL...,"ACT_SITE 85; /evidence=""ECO:0000305|PubMed:78...","BINDING 30; /note=""Substrate""; /evidence=""EC..."
2,Q0VCA5,SAMH1_BOVIN,reviewed,Deoxynucleoside triphosphate triphosphohydrola...,SAMHD1,Bos taurus (Bovine),589,MQSADSQNTPKRPRRDGSPRTPPDSPLADAETSPSHDLDPDYRTWG...,"ACT_SITE 221; /evidence=""ECO:0000250|UniProtK...","BINDING 104; /note=""GTP""; /evidence=""ECO:000..."
3,O45539,SRC2_CAEEL,reviewed,Tyrosine protein-kinase src-2 (EC 2.7.10.2) (S...,src-2 kin-22 F49B2.5,Caenorhabditis elegans,507,MGSCIGKEDPPPGATSPVHTSSTLGRESLPSHPRIPSIGPIAASSS...,"ACT_SITE 358; /note=""Proton acceptor""; /evid...","BINDING 268; /note=""ATP""; /evidence=""ECO:000..."
4,Q27324,RYK1_DROME,reviewed,Tyrosine-protein kinase Drl (EC 2.7.10.1) (Pro...,drl lio CG17348,Drosophila melanogaster (Fruit fly),610,MAPNLLTIGLLLTLIASGQAHLNIFLNLHEVLRLIGVSAELYYVRE...,"ACT_SITE 468; /note=""Proton acceptor""; /evid...","BINDING 371; /note=""ATP""; /evidence=""ECO:000..."


In [7]:
df.loc[:, 'Active site'] = df.loc[:, 'Active site'].apply(get_sites)
df.loc[:, 'Binding site'] = df.loc[:, 'Binding site'].apply(get_binding_sites)

In [8]:
df[:5]

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Sequence,Active site,Binding site
0,Q9SJT1,SAE2_ARATH,reviewed,SUMO-activating enzyme subunit 2 (EC 2.3.2.-) ...,SAE2 EMB2764 At2g21470 F3K23.23,Arabidopsis thaliana (Mouse-ear cress),700,MATQQQQSAIKGAKVLMVGAGGIGCELLKTLALSGFEDIHIIDMDT...,[168],"[43, 67]"
1,P56221,SCYD_MAGO7,reviewed,Scytalone dehydratase (SD) (SDH) (EC 4.2.1.94),SDH1 MGG_05059,Magnaporthe oryzae (strain 70-15 / ATCC MYA-46...,172,MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTL...,"[85, 110]","[30, 50, 53, 131]"
2,Q0VCA5,SAMH1_BOVIN,reviewed,Deoxynucleoside triphosphate triphosphohydrola...,SAMHD1,Bos taurus (Bovine),589,MQSADSQNTPKRPRRDGSPRTPPDSPLADAETSPSHDLDPDYRTWG...,[221],"[104, 107, 137, 152, 198, 304, 308, 322, 347, ..."
3,O45539,SRC2_CAEEL,reviewed,Tyrosine protein-kinase src-2 (EC 2.7.10.2) (S...,src-2 kin-22 F49B2.5,Caenorhabditis elegans,507,MGSCIGKEDPPPGATSPVHTSSTLGRESLPSHPRIPSIGPIAASSS...,[358],[268]
4,Q27324,RYK1_DROME,reviewed,Tyrosine-protein kinase Drl (EC 2.7.10.1) (Pro...,drl lio CG17348,Drosophila melanogaster (Fruit fly),610,MAPNLLTIGLLLTLIASGQAHLNIFLNLHEVLRLIGVSAELYYVRE...,[468],[371]


# Load into unirep mlstm

In [9]:
import tensorflow as tf

E:\Academic\School\SPRING2020\CS294-150\project\294env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Academic\School\SPRING2020\CS294-150\project\294env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Academic\School\SPRING2020\CS294-150\project\294env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Academic\Scho

In [11]:
import os
from UniRep.unirep import babbler1900 as babbler

W0427 12:51:11.424242  1000 deprecation_wrapper.py:119] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\unirep.py:43: The name tf.nn.rnn_cell.RNNCell is deprecated. Please use tf.compat.v1.nn.rnn_cell.RNNCell instead.

W0427 12:51:14.463926  1000 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [12]:
tf.reset_default_graph()

In [13]:
# If you want to initialize a model from scratch, set new to True and trained to False
batch_size = 12
b = babbler(batch_size=batch_size, model_path=f'2000_iters_hs256_nl4/', trained=True, 
                rnn_size=256, n_layers=4, new=False)

W0427 12:52:39.030457  1000 deprecation_wrapper.py:119] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\unirep.py:336: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0427 12:52:39.463955  1000 deprecation_wrapper.py:119] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\unirep.py:390: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0427 12:52:39.472929  1000 deprecation.py:323] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\unirep.py:400: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0427 12:52:39.883568  1000 deprecation_wrapper.py:119] From E:\Academic\School\SPRING2020\CS294-150\project\294env\lib\site-packages\tensorflow\python\autograph\converters\directives.py:117: The name tf.variable_scope is deprecated. Please use tf

In [14]:
# get sequences of length less than 275
df_le = df[df['Length']<275]
# get site sequences, 0=normal, 1=active, 2=binding
sequences = []
for i,row in df_le.iterrows():
    seq_len = row['Length']
    site_seq = np.zeros(seq_len+1,dtype=np.int32)
    site_seq[np.array(row['Active site'], dtype=np.int32)] = 1
    site_seq[np.array(row['Binding site'], dtype=np.int32)] = 2
    sequences.append(site_seq)
df_le['Site sequence'] = sequences

c:\users\fores\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [15]:
df_le.iloc[0]

Entry                                                       P56221
Entry name                                              SCYD_MAGO7
Status                                                    reviewed
Protein names       Scytalone dehydratase (SD) (SDH) (EC 4.2.1.94)
Gene names                                          SDH1 MGG_05059
Organism         Magnaporthe oryzae (strain 70-15 / ATCC MYA-46...
Length                                                         172
Sequence         MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTL...
Active site                                              [85, 110]
Binding site                                     [30, 50, 53, 131]
Site sequence    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 1, dtype: object

In [16]:
for entry in df_le[:5].iterrows():
    print(entry[1]['Sequence'])

MGSQVQKSDEITFSDYLGLMTCVYEWADSYDSKDWDRLRKVIAPTLRIDYRSFLDKLWEAMPAEEFVGMVSSKQVLGDPTLRTQHFIGGTRWEKVSEDEVIGYHQLRVPHQRYKDTTMKEVTMKGHAHSANLHWYKKIDGVWKFAGLKPDIRWGEFDFDRIFEDGRETFGDK
MAIKLIAIDMDGTLLLPDHTISPAVKNAIAAARARGVNVVLTTGRPYAGVHNYLKELHMEQPGDYCITYNGALVQKAADGSTVAQTALSYDDYRFLEKLSREVGSHFHALDRTTLYTANRDISYYTVHESFVATIPLVFCEAEKMDPNTQFLKVMMIDEPAILDQAIARIPQEVKEKYTVLKSAPYFLEILDKRVNKGTGVKSLADVLGIKPEEIMAIGDQENDIAMIEYAGVGVAMDNAIPSVKEVANFVTKSNLEDGVAFAIEKYVLN
MEGQRWLPLEANPEVTNQFLKQLGLHPNWQFVDVYGMEPELLSMVPRPVCAVLLLFPITEKYEVFRTEEEEKIKSQGQDVTSSVYFMKQTISNACGTIGLIHAIANNKDKMHFESGSTLKKFLEESVSMSPEERAKFLENYDAIRVTHETSAHEGQTEAPSIDEKVDLHFIALVHVDGHLYELDGRKPFPINHGKTSDETLLEDAIEVCKKFMERDPDELRFNAIALSAA
MLRLLVVASLVLYGHSTQDFPETNARVVGGTEAQRNSWPSQISLQYRSGSSWAHTCGGTLIRQNWVMTAAHCVDRELTFRVVVGEHNLNQNDGTEQYVGVQKIVVHPYWNTDDVAAGYDIALLRLAQSVTLNSYVQLGVLPRAGTILANNSPCYITGWGLTRTNGQLAQTLQQAYLPTVDYAICSSSSYWGSTVKNSMVCAGGDGVRSGCQGDSGGPLHCLVNGQYAVHGVTSFVSRLGCNVTRKPTVFTRVSAYISWINNVIASN
MAKNRFNQHWLHDHINDPYVKMAQREGYRARAAYKLKEIDEQDKLIRPGQVIVDLGAT

In [17]:
# Before you can train your model, 
with open("formatted.txt", "w") as destination:
    for i,row in df_le.iterrows():
        seq = row['Sequence']
        site_seq = row['Site sequence']
        if b.is_valid_seq(seq): 
            seq = b.format_seq(seq)
            interleaved_seq = np.zeros(2*len(seq), dtype=np.int32)
            # interleave the amino acid sequence and site sequence
            # because this is how we are storing both datas
            interleaved_seq[::2] = seq
            interleaved_seq[1::2] = site_seq+1
            formatted = ",".join(map(str,interleaved_seq))
            destination.write(formatted)
            destination.write('\n')

In [18]:
bucket_op = b.bucket_batch_pad("formatted.txt", lower=200, interval=1000) # Large interval

W0427 12:53:07.163650  1000 deprecation_wrapper.py:119] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\data_utils.py:94: The name tf.string_to_number is deprecated. Please use tf.strings.to_number instead.

W0427 12:53:07.178602  1000 deprecation.py:323] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\data_utils.py:190: group_by_window (from tensorflow.contrib.data.python.ops.grouping) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.experimental.group_by_window(...)`.
W0427 12:53:07.210974  1000 deprecation.py:323] From E:\Academic\School\SPRING2020\CS294-150\project\UniRep\unirep.py:601: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use 

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    em = sess.run(b.embed_matrix)
   
# even indices = amino acid 
# odd indices = site type
print(batch)
print(batch.shape)

[[24  1  1 ...  0  0  0]
 [24  1  1 ...  0  0  0]
 [24  1  1 ...  0  0  0]
 ...
 [24  1  1 ...  0  0  0]
 [24  1  1 ...  0  0  0]
 [24  1  1 ...  0  0  0]]
(12, 530)


In [20]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

In [21]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)/2
    return lengths

nonpad_len(batch)

array([173., 243., 150., 265., 247., 256., 232., 202., 256., 138., 260.,
       110.])

In [22]:
optimizer = tf.train.AdamOptimizer(0.001)
all_step_op = optimizer.minimize(b._loss+b.site_loss)

W0427 12:53:11.000681  1000 deprecation.py:323] From E:\Academic\School\SPRING2020\CS294-150\project\294env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [26]:
dummy='dummy_folder'

In [27]:
if not os.path.exists(dummy):
    os.mkdir(dummy)

In [28]:
# How training looks like
num_iters = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        seq_batch = batch[:, ::2]
        site_seq_batch = batch[:, 1::2]-1
        length = nonpad_len(batch)-1
        loss, site_loss, _= sess.run([b._loss, b.site_loss, all_step_op],
                                       feed_dict = {
                                        x_placeholder: seq_batch[:, :-1],
                                        b._minibatch_y_placeholder: seq_batch[:, 1:],
                                        b._minibatch_site_y_placeholder: site_seq_batch[:, :-1],
                                        initial_state_placeholder: b._zero_state,
                                        seq_length_placeholder: length,
                                        batch_size_placeholder: batch_size,
                                        b._temp_placeholder: 0.1
                                       })
    
        print(f'Iteration {i}, loss {loss}, site loss {site_loss}')
    b.dump_weights(sess, dummy)

Iteration 0, loss 1.8776484727859497, site loss 0.010596365667879581
embed_matrix_0
[[ 0.01795816 -0.59574187 -0.31995562  0.06984629  0.40433308 -0.20387918
   0.20837028  0.02349814 -0.08639826  0.3730955 ]
 [ 0.5577035  -0.7487646  -0.10239369 -0.32437634 -0.5262135  -0.15576366
   0.4926013  -0.17384009  0.40252152 -0.21631326]
 [ 0.5650982   0.23117451  0.40731284 -0.3695129   0.23110402  0.1681975
  -0.02075569 -0.47987574 -0.66583145 -0.00447276]
 [-0.03245609 -0.11626884 -0.33265555  0.5045284  -0.17571148  0.05205186
   0.16283442  0.19018494 -0.12135593 -0.49977335]
 [-0.5703836  -0.446131    0.2960824  -0.1666478  -0.31925482 -0.12011106
   0.02298783  0.02474621 -0.18123847 -0.32975945]
 [ 0.33877403  0.72887784 -0.26684377 -0.53426933  0.06602486 -0.26432738
   0.21926865  0.38794106 -0.46740597 -0.15146168]
 [-0.68923706 -0.31945857  0.13209684 -0.3690833   0.3801327   0.00466809
   0.29135716 -0.17630763 -0.33551458  0.38000694]
 [ 0.05775635 -0.04033554  0.5785264  -0.3

rnn/mlstm_stack/mlstm_stack1/mlstm_stack1/wh_0
[[ 0.03058092  0.07574227 -0.04154119 ...  0.01504024 -0.16453116
  -0.05265732]
 [ 0.11254816 -0.1757097  -0.05270271 ...  0.20333765  0.05326265
   0.07906359]
 [-0.07605886 -0.10128421 -0.04283068 ...  0.03830232  0.06412466
   0.10309768]
 ...
 [ 0.04668658  0.0159542  -0.01361777 ... -0.04152701 -0.0678052
   0.06222383]
 [ 0.11282471  0.01528359  0.10479821 ...  0.05632594 -0.02272473
   0.13245209]
 [-0.08332232 -0.05206282 -0.0748313  ... -0.01816005  0.01620666
   0.05192275]]
rnn/mlstm_stack/mlstm_stack1/mlstm_stack1/wmx_0
[[ 0.09421043 -0.0781231   0.02037435 ... -0.02504922  0.04252028
   0.03068395]
 [ 0.21274836  0.00855812 -0.10528336 ... -0.12454712  0.04522993
   0.03402837]
 [ 0.02699681 -0.1573665  -0.15890574 ... -0.02753062  0.02302762
  -0.08012205]
 ...
 [ 0.02670928 -0.17337579 -0.13450345 ... -0.08739836 -0.17312494
   0.03137153]
 [ 0.03731266 -0.06851152 -0.20184967 ...  0.12788942  0.14036721
  -0.13469648]
 [ 0

rnn/mlstm_stack/mlstm_stack2/mlstm_stack2/b_0
[0.9869088  1.0195531  1.0045556  ... 1.0151227  0.97398984 1.0394554 ]
rnn/mlstm_stack/mlstm_stack2/mlstm_stack2/gx_0
[1.0722752  1.0518187  0.985807   ... 1.028929   1.05064    0.98292613]
rnn/mlstm_stack/mlstm_stack2/mlstm_stack2/gh_0
[1.0500592 1.055963  1.0372034 ... 1.0095779 0.9779369 1.0382587]
rnn/mlstm_stack/mlstm_stack2/mlstm_stack2/gmx_0
[1.0519857 1.1069614 1.1262553 1.1349046 1.0496802 1.0689726 1.1137491
 1.1090862 1.0589852 1.0463536 1.1260011 1.1515509 1.1284994 1.0571102
 1.0953683 1.1648903 1.1232973 1.0794948 1.1697711 1.1325263 1.0762184
 1.1306193 1.1007996 1.0926807 1.1552886 1.0496541 1.0528674 1.0475335
 1.0768211 1.0697201 1.1276075 1.0508606 1.1463007 1.1190034 1.199441
 1.0428331 1.0599384 1.1503233 1.1236423 1.1547949 1.0737019 1.1188691
 1.0943556 1.0817437 1.0898468 1.1075395 1.1597595 1.058198  1.1044289
 1.0651953 1.2097456 1.1109205 1.1058978 1.0954901 1.1396918 1.056434
 1.0520744 1.1367759 1.1541733 1.053

fully_connected/biases_0
[ 0.03505907  0.01039774 -0.0016424   0.00355181  0.00422901  0.00458979
  0.00787218  0.01096485  0.00654174  0.0100566  -0.0129068  -0.04903596
  0.00998564  0.00271189  0.00904888  0.0086786   0.00993572  0.00210927
  0.00435306  0.00063221  0.00733471 -0.12081234 -0.11099907 -0.11211418
 -0.10853834]
fully_connected_1/weights_0
[[ 1.33566245e-01  2.82388311e-02 -7.35270604e-02]
 [ 5.80520295e-02 -2.11535886e-01  9.84705761e-02]
 [-1.01594336e-01 -1.21100783e-01  1.24244280e-01]
 [ 2.36091018e-02  4.29169051e-02  1.67265534e-01]
 [ 2.19468430e-01 -2.26097852e-01 -1.06713008e-02]
 [-7.74747431e-02  6.90045804e-02 -2.20922709e-01]
 [-1.56648472e-01 -1.65638775e-02  8.58242735e-02]
 [-7.15215281e-02 -5.55984713e-02 -1.10751152e-01]
 [ 1.46027029e-01  1.28295064e-01  4.34653535e-02]
 [-8.48647207e-02  1.28462866e-01  1.09572962e-01]
 [-1.33235669e-02 -1.21733449e-01 -6.63605034e-02]
 [-2.13183463e-01  3.13877910e-01  6.93300217e-02]
 [ 1.37540758e-01 -5.73109575

## Get the representation of an amino acid sequence

In [29]:
# tensorflow is kinda dumb so you need to do this whenever you initialize a new model 
# of the same type, so if you want to rereun the previous stuff after initializing 
# b_trained you need to go all the way back to the beginning where it also resets the 
# default graph
tf.reset_default_graph()

In [30]:
# set batch size to one to predict next character for one sequence at a time
# in next section
b_trained = babbler(batch_size=1, model_path=f'./2000_iters_hs256_nl4/', trained=True,
                   rnn_size=256, n_layers=4, new=False)

In [31]:
i=np.random.randint(0, 10000)
seq = df.iloc[i]['Sequence']
df.iloc[i]

Entry                                                       B2K830
Entry name                                              ACKA_YERPB
Status                                                    reviewed
Protein names            Acetate kinase (EC 2.7.2.1) (Acetokinase)
Gene names                                          ackA YPTS_2691
Organism         Yersinia pseudotuberculosis serotype IB (strai...
Length                                                         400
Sequence         MSSKLVLVLNCGSSSLKFAIIDATNGEEHISGLAECFHLPEARIKW...
Active site                                                  [150]
Binding site                                              [17, 91]
Name: 4680, dtype: object

In [32]:
# how to obtain hidden representations of MLSTM
avg_hidden, final_hidden, final_cell = b_trained.get_rep(seq)

## see how well it does on this sequence

In [33]:
batch = np.array([np.array(b_trained.format_seq(seq))])

In [34]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b_trained.get_rep_ops())

In [35]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    length = batch[0].shape[0]-1
    loss, logits, sample = sess.run([b_trained._loss, b_trained._logits, b_trained._sample],
                                   feed_dict = {
                                    x_placeholder: batch[:, :-1],
                                    b_trained._minibatch_y_placeholder: batch[:, 1:],
                                    initial_state_placeholder: b_trained._zero_state,
                                    seq_length_placeholder: np.array([length]),
                                    batch_size_placeholder: 1,
                                    b_trained._temp_placeholder: 0.1
                                   })

    print(f'loss {loss}')

# for reference, uniform random predictor would get ln(24) = 3.17. 

loss 2.7328014373779297


In [36]:
# 2nd through nth characters of sequence
aas=batch[0][1:]
aas

array([ 1,  7,  7,  4, 21, 16, 21, 16, 21,  9, 11, 13,  7,  7,  7, 21,  4,
       18, 15, 17, 17,  5, 15,  8,  9, 13,  6,  6,  3, 17,  7, 13, 21, 15,
        6, 11, 18,  3, 21, 14,  6, 15,  2, 17,  4, 20,  4, 16,  5, 13, 13,
        4, 10,  6, 15, 15, 21, 13, 15, 13, 15, 15,  3,  7,  6, 15, 21,  9,
       18, 17, 16,  9,  8, 17, 21, 15, 10,  4, 14, 15, 21,  7, 15, 10, 21,
        8, 15, 17, 13,  3,  2, 17, 16,  3, 13, 13,  6,  4, 18,  8,  7,  7,
       16, 17, 16,  8,  6,  5, 16, 17, 10, 13, 17,  4,  5,  7, 17, 14, 18,
       15, 14, 21,  3,  9, 14, 15,  3, 21, 17, 13, 17, 15,  6, 15, 21,  4,
        7, 18, 14,  9, 21, 15,  5,  4,  9, 16, 15, 16, 18,  5,  8, 15, 18,
        3, 10,  8,  1, 14,  6,  6,  7, 19, 21, 19, 15, 21, 14, 19,  7, 21,
       19,  4,  5,  3, 13, 17,  2,  2, 19, 13, 15,  3, 13,  8,  7,  3, 18,
       19, 16,  7, 10,  6, 15, 15,  4, 17, 21,  9,  4, 14, 21, 13,  6, 21,
        9, 16, 17,  8, 11,  3, 21, 13,  9, 13, 13,  7, 16,  8, 15, 16,  2,
        9, 13,  4, 11, 16

In [37]:
# predicted 2nd through nth characters of sequence
aas2=sample[0]+1
aas2

array([ 1,  7,  8, 21,  8,  8, 16, 16,  9,  9, 13, 13,  7,  7,  7,  7,  4,
       21, 10, 21,  6,  6, 15, 21,  4,  4, 15,  4, 16, 17, 16, 15, 15, 15,
       15, 21, 15,  6, 15,  7,  6,  9, 15, 15, 15, 15, 21,  6,  6, 21,  6,
       15, 16, 16, 16, 16, 21, 15, 15,  7, 13, 17, 16, 21, 21, 15, 15, 15,
       21, 21, 15,  5, 13,  8,  6,  5,  5,  5, 17,  5, 15, 16, 13,  4, 15,
       15,  5,  6, 15, 15, 15, 17,  5,  5, 13, 15,  6,  7, 15, 15, 13,  6,
       21, 15,  6, 15, 13, 15, 21, 15,  6, 15, 15, 15, 15, 16, 16, 17, 16,
       16, 16,  6, 15, 16, 15,  5, 21, 21,  7, 15, 15, 13,  7, 13, 15, 15,
       15, 15, 13,  6, 16, 16, 17, 21, 17, 17, 17, 16, 16,  8, 15, 15, 18,
        3, 13,  8, 19, 13,  6, 15, 15, 21, 15, 21, 15, 21,  6,  6, 16,  6,
       15, 13,  6, 16, 17, 16, 16, 16, 16, 15, 16, 21, 15,  7, 16, 21, 21,
       15, 15, 15, 13, 15, 21,  4,  4, 15, 15,  4,  6,  6, 21, 21, 16, 17,
       16, 21, 16, 13, 13,  7,  8, 13, 13,  7,  9, 15, 21, 21, 21, 15, 15,
        6, 15, 15, 16, 16

In [38]:
print(f"Proportion of amino acids predicted correctly: {np.sum(aas==aas2)/len(aas)}")

Proportion of amino acids predicted correctly: 0.135
